In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%%html

<style>
    .jp-OutputArea-child {
        display: inline-block;
    }
</style>

In [ ]:
#!conda install -y pillow
#!conda install -y -c conda-forge opencv

In [ ]:
import sys
sys.path.append("..")

In [ ]:
import inochi2d.io
import json
import io
import cv2
from PIL import Image, ImageDraw
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import HTML,JSON

In [ ]:
#puppet_aka = inochi2d.io.load("/home/seagetch/ドキュメント/demon-lord-20220718-1.6.1.inx")
#puppet_aka = inochi2d.io.load("/home/seagetch/ドキュメント/girl-summer-suimwear-1.4.inx")
puppet_aka = inochi2d.io.load("../Aka-test.inx")

In [ ]:
def show(name, tex):
    if tex is None:
        display(name)
    elif tex.width > tex.height:
        display(name, tex.resize((64, int(64 / tex.width * tex.height))))
    else:
        display(name, tex.resize((int(64 / tex.height * tex.width), 64)))

In [ ]:
STEP = 128
STEP_D = 32
for node in puppet_aka.nodes():
    img = node.textures()
    if img is None:
        continue
    img = img[0]
    size = max(img.width, img.height)
    img_scale = 1
    
    step = 1
    if size < 256:
        step = 256 / size * 4
    img2 = img.resize((int(img.width / img_scale), int(img.height / img_scale)), Image.LANCZOS)
    img2 = np.array(img2, dtype=np.uint8)
    mask = img2[:,:,3]
    img3 = cv2.cvtColor(img2, cv2.COLOR_RGB2BGR)
    ret, mask = cv2.threshold(mask, 15,255,0)
#    kernel = np.ones((5,5),np.uint8)
#    mask = cv2.erode(mask,kernel,iterations = 5)
#    mask = cv2.dilate(mask,kernel,iterations = 5)
    contours, hierarchy = cv2.findContours(mask,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
    l_contours = [len(c) for c in contours]
    if len(l_contours) < 1:
        continue
    max_cont = max(l_contours)

    def calc_moment(vertices):
        if len(vertices) == 0:
            return
        cx = int(img.width / 2)
        cy = int(img.height / 2)
        return vertices - np.array([cx, cy])
        
    
    def scaling(_cont, scale):
        try:
            M = cv2.moments(_cont)
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
        except ZeroDivisionError as e:
            cx = 0
            cy = 0
        cnt_norm = _cont - [cx, cy]
        cnt_scaled = cnt_norm * scale
        cnt_scaled = cnt_scaled + [cx, cy]
        cnt_scaled = cnt_scaled.astype(np.int32)
        return cnt_scaled


    _img3 = img3.copy()
    vertices = []
    for i in range(len(contours)):
        _cont =contours[i]
        if len(_cont) < 10:
            continue
        _cont = _cont#[::STEP]
        
        def resampling (_cont, STEP):
            _cont2 = [_cont[0]]
            prev=_cont[0]
            for _c in _cont:
                if np.linalg.norm(prev - _c) > STEP:
                    _cont2.append(_c)
                    prev = _c
            return np.array(_cont2)
        
        for scale in [1.10, 1, 0.95, 0.8, 0.6, 0.4, 0.2]:
            __cont1 = resampling(_cont, STEP_D / scale / scale /step)
            __cont2 = resampling(_cont, STEP_D / scale / scale /step)
            if scale < 0.7:
                _cont = __cont2
            else:
                _cont = __cont1
            cnt_scaled = scaling(_cont, scale)
            prev = None
            for v in cnt_scaled:
                v = v[0]
                dist = min([np.linalg.norm(u-v) for u in vertices]) if len(vertices) > 0 else 1
                if dist > 0.1:
                    vertices.append(v)

    if len(vertices) < 1:
        continue
    minx = min([p[0] for p in vertices])
    miny = min([p[1] for p in vertices])
    maxx = max([p[0] for p in vertices])
    maxy = max([p[1] for p in vertices])
    w = maxx - minx
    h = maxy - miny
    sd = cv2.Subdiv2D((0,0,w,h))
    for i,p in enumerate(vertices):
        try:
            p2 = p.astype("f")-np.array((minx, miny))
            sd.insert((p2[0], p2[1]))
        except Exception as e:
            pass
    tri = sd.getTriangleList().reshape(-1, 2)
    for i in range(tri.shape[0]):
        tri[i] += (minx, miny)
    tri = tri.reshape(-1, 3, 2)
    cv2.polylines(_img3, tri.astype(int), True, (255, 0, 0), thickness=2)
    for v in vertices:
        cv2.drawMarker(_img3, (int(v[0]), int(v[1])),(0, 255, 0), thickness=2)
    img4 = cv2.cvtColor(_img3, cv2.COLOR_BGR2RGB)
    img4 = Image.fromarray(img4)
    img4 = img4.resize((int(img4.width / 4), int(img4.height / 4)))
    
    flat_indices = []
    vertset = {}
    for i,v in enumerate(vertices):
        vertset[(int(v[0]), int(v[1]))] = i
    for t in tri:
        t1 = t[0].astype(int)
        t2 = t[1].astype(int)
        t3 = t[2].astype(int)
        id1 = vertset.get((t1[0], t1[1]))
        id2 = vertset.get((t2[0], t2[1]))
        id3 = vertset.get((t3[0], t3[1]))
        if id1 is not None and id2 is not None and id3 is not None:
            flat_indices.extend([id1, id2, id3])
    print("%d/%d"%(len(tri), len(flat_indices)/3))

    display(img4)
    flat_vertices = []
    flat_uvs = []
    if len(vertices) > 0:
        for v in calc_moment(np.array(vertices)):
            flat_vertices.extend(v.tolist())
            flat_uvs.extend([v[0] / img.width + 0.5, v[1] / img.height + 0.5])
            
    node.root["mesh"]["verts"] = flat_vertices
    node.root["mesh"]["uvs"]=flat_uvs
    node.root["mesh"]["indices"] = flat_indices

In [ ]:
puppet_aka.clear_binding()

In [ ]:
JSON([n.root for n in puppet_aka.nodes(name="Eyelid")])

In [ ]:
#raw_data = inochi2d.io.dump_inp(puppet)
inochi2d.io.dump("../Aka-test2.inx", puppet_aka)